In [2]:
import duckdb

import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import utils 

## Join two sources into one table

In [10]:

nationalites = utils.DBtable_to_df('../../integration.duckdb', 'nationalitiesClean_Madrid')
households = utils.DBtable_to_df('../../integration.duckdb','householdClean_Madrid')

result = households.merge(nationalites, how='inner', on=["Madrid_section","Year"])

utils.df_to_DBtable('../../integration.duckdb',result, 'integratedTable')

### Test

In [13]:
# check if all tables in integration.duckdb are created
utils.get_tables('../../integration.duckdb')

## Removing redundacies from integrated table

### Removing columns with zeros in all rows

In [7]:
integratedDF = utils.DBtable_to_df('../integration.duckdb', 'integratedTable')
for (columnName, columnData) in integratedDF.items():
    if(integratedDF[columnName] == 0).all():
        integratedDF.drop(columnName, axis=1, inplace=True)
        print(columnName)
display(integratedDF)


CatalogException: Catalog Error: Table with name integratedTable does not exist!
Did you mean "pg_tables"?
LINE 1: SELECT * FROM integratedTable
                      ^

### Removing aggregated columns 'suma de categorías de nacionalidad' and 'population'

In [ ]:
integratedDF.drop(['suma de categorías de nacionalidad', 'population'], axis=1, inplace=True)
display(integratedDF)


### Checking if there is section code with all zeros in remaining columns

In [ ]:

display(integratedDF[integratedDF.columns[1:]].loc[(integratedDF[integratedDF.columns[1:]]== 0).all(axis=1)])
# none are found



# Feature engineering

Groupping nationalities that have less than 0.5% of the total population (from 2018 to 2020) in a feature called Others.

In [43]:
df = utils.DBtable_to_df('../../integration.duckdb', 'integratedTable')
df.drop(['suma de categorías de nacionalidad', 'population', 'Extranjeros'], axis=1, inplace=True)

In [57]:
cols = list(range(22, df.shape[1]))
cols[0] = df.columns.get_loc('Madrid_section') # nationalities - year- madrid section - extranjeros - total nacionalidades

nat = df.iloc[:,cols]
nat.set_index('Madrid_section', inplace = True)
nat.astype('int64')

,Year,Afganistán,Albania,Alemania,Andorra,Angola,Antigua_y_Barbuda,Arabia_Saudita,Argelia,Argentina,...,Ucrania,Uganda,Uruguay,Uzbekistán,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
Madrid_section,,,,,,,,,,,,,,,,,,,,,
079601001,2018,0,0,2,0,0,0,0,0,2,...,0,0,0,0,0,6,0,0,0,0
079601002,2018,0,0,3,0,0,0,0,0,5,...,0,0,3,0,0,9,0,0,0,0
079601003,2018,0,0,5,0,0,0,0,2,15,...,6,0,5,0,0,21,0,0,0,0
079601004,2018,0,0,7,0,0,0,0,1,3,...,0,1,2,0,0,14,0,0,0,0
079601006,2018,0,0,18,0,1,0,0,0,8,...,5,0,2,0,0,13,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
079621029,2019,0,0,3,0,0,0,0,0,4,...,0,0,0,0,0,3,0,0,0,0
079621030,2019,0,0,2,0,0,0,0,0,2,...,6,0,0,0,0,10,0,0,0,0
079621031,2019,0,0,3,0,0,0,0,0,3,...,4,0,0,0,0,36,0,1,0,0


In [69]:
nat.loc['Total (%)'] = round((nat.iloc[:,1:].sum()/(nat.iloc[:,1:].sum()).sum()) * 100,3)

natDef = (nat.loc[:, nat.loc['Total (%)'] >= 0.5])
natDef['Others'] = (nat.loc[:, nat.loc['Total (%)'] < 0.5]).sum(axis = 1)
natDef['Year'] = nat.Year

natDef.drop('Total (%)', inplace = True)

/tmp/ipykernel_18132/1006348694.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  natDef['Others'] = (nat.loc[:, nat.loc['Total (%)'] < 0.5]).sum(axis = 1)
/tmp/ipykernel_18132/1006348694.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  natDef['Year'] = nat.Year
/tmp/ipykernel_18132/1006348694.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [70]:
integratedDF = households.merge(natDef, how='inner', on=["Madrid_section","Year"])

### Updating integrated table 

In [71]:
utils.df_to_DBtable('../../integration.duckdb',integratedDF, 'integratedTable')

## Saving copy of integrated table to analysis folder, together with utils needed for analysis

In [72]:
import shutil

shutil.copy(os.path.join("../integration.duckdb"), os.path.join("../analysis/"))
shutil.copy(os.path.join("../utils.py"), os.path.join("../analysis/"))

print('Copied')

Copied


: 